# Improve Text Retrieval Using Hybrid Search with BGE-M3

BGE-M3 has been introduced to the field of natural language processing and information retrieval. This model is remarkable for its multilingual capabilities, allowing it to support a wide array of languages. Moreover, BGE-M3 is capable of producing various types of embeddings, a feature that has been empirically proven to enhance retrieval effectiveness through hybrid search methodologies.     
Hybrid search, which combines multiple types of embeddings to conduct searches, has been shown to significantly improve performance. This approach leverages the strengths of different embedding types to achieve higher accuracy and more relevant search results.   
<img src="./misc/table2.png" alt="Sample Image" title="Sample Image" width="75%" />     


We will explore how to utilize the BGE-M3 model to implement a hybrid search on the MIRACL Finnish dataset using Milvus 2.4.0, aiming to demonstrate the performance improvements that can be achieved. 


## Prerequisite
Checkout [Milvus official website](https://milvus.io/docs/install_standalone-docker.md) to install milvus>=2.4.0. And install PyMilvus and other dependencies by 
```bash
!pip install -r requirements.txt
```

## Generate Embeddings of the Corpus
First and foremost, we need to generate embeddings for the Finnish language. To assist you in this task, we've prepared a script designed to transform the corpus of your chosen language from MIRACL into vectors, which will then be stored in the `embeddings` folder. Given the substantial size of the MIRACL dataset, we recommend conducting this operation on a machine equipped with a GPU to facilitate efficient computation. Upon completion of this process, you will have successfully obtained the corpus  embeddings.  
Additionally, considering the vast quantity of data to be processed, we advise against running this script directly within this notebook. Instead, we recommend executing it from the command line to ensure optimal performance and manageability.

In [ ]:
!python generate_embeddings.py

## Insert Embeddings to Milvus 
Now, the `embeddings` folder contains the Finnish corpus embeddings. To proceed with searching, we will insert them into Milvus. Assuming your Milvus installation is correctly set up, once the script completes execution, each sentence's dense embeddings and sparse embeddings will have been inserted into the vector database, and indexes will have been created accordingly.

In [ ]:
!python insert.py

## Generate Search Results

Now, we utilize queries to search through these corpus embeddings. Here, we employ a WeightedRanker to combine the Inner Product (IP) scores of dense embeddings with those of sparse embeddings through weighting. In this equation, alpha represents the weight for the similarity of dense embeddings, and beta represents the weight for the similarity of sparse embeddings. This means the score will be a hybrid one, calculated as:  
<div style="text-align: center;">
    <img src="./misc/formula.png" alt="Sample Image" title="Sample Image" width="45%" />
</div>
We will generate three sets of data, specifically: results using only dense embeddings, results using only sparse embeddings, and results from a certain ratio of mixed embeddings.

In [ ]:
!python search.py --dense 1.0 --sparse 0.0 --output results_dense.json
!python search.py --dense 0.0 --sparse 1.0 --output results_sparse.json
!python search.py --dense 1.0 --sparse 0.4 --output results_hybrid.json


## Evaluate the Results
We have now generated three JSON files. For those who skipped the steps above, we have prepared these three JSON files for evaluation purposes. Let's now take a look at the NDCG@10 metric based on them.








In [1]:
from utils import eval_miracl

eval_miracl('fi', './results_dense.json')

./results_dense.json NDCG@10: 0.78556466046342



This corresponds to the score of 78.6 in the 'fi' column under the 'dense' row of the result table, demonstrating that BGE-M3's dense model possesses quite powerful performance.

In [2]:
from utils import eval_miracl

eval_miracl('fi', './results_sparse.json')

./results_sparse.json NDCG@10: 0.6554212963439118


This corresponds to the score of 65.3 in the 'fi' column under the 'sparse' row of the result table, demonstrating the outstanding performance of BGE-M3 sparse and even showcasing some strong dense model baselines.

In [3]:
from utils import eval_miracl

eval_miracl('fi', './results_hybrid.json')

./results_hybrid.json NDCG@10: 0.7961152130482131



This corresponds to the score of 79.7 in the 'fi' column under the 'dense+sparse' row of the result table, demonstrating the further enhancement brought to the search results by combining dense embeddings with sparse embeddings.







## Conclusion
Ultimately, by utilizing BGE-M3 for Finnish, we achieved superior results through hybrid search, opening up a new perspective. This approach, which combines different vector models, yields enhanced outcomes. Of course, identifying the optimal hybrid method requires experimentation to find the most effective strategy. Besides quantitative metrics reflecting improvement, the use of lexical models and dense models can also produce unexpectedly positive results in scenarios that demand precise matching.